In [2]:
import os 
import numpy as np 
import cv2 
from tqdm import tqdm 
import json
import math 
import shutil
import argparse
import glob
from os.path import basename, splitext
from pickletools import uint8

import geopandas as gp
import numpy as np
import pandas as pd
from shapely.geometry import *
import os.path
from PIL import Image

## 1. Create Mask Image

In [3]:
ZOOM_LEVEL = '18'
PARENT_DIR = 'data/test/output'
INPUT_DIR = f'{PARENT_DIR}/{ZOOM_LEVEL}'
OUTPUT_DIR = f'{PARENT_DIR}/{ZOOM_LEVEL}_all/'
IMG_NAME = 'prediction_gan_18'

### 1.1 Merge all tiles from different folder into a folder

<zoom_level>/<x>/{y}.png -> <single folder>/{zoom_level}_{x}_{y}.png

In [11]:
for _, x in enumerate(os.listdir(INPUT_DIR)):
    for y in os.listdir(os.path.join(INPUT_DIR, x)):
        shutil.copy(os.path.join(INPUT_DIR, x, y),
                    os.path.join(OUTPUT_DIR, f'{ZOOM_LEVEL}_{x}_{y}'))
        
# Merge all tile image into a single image

img_list = {}
for img_file in tqdm(sorted(os.listdir(OUTPUT_DIR)), total=len(os.listdir(OUTPUT_DIR))):
    zoom_level, x, y = img_file.replace('.png', '').strip().split('_')
    if x not in img_list:
        img_list[x] = []
    img = cv2.imread(os.path.join(OUTPUT_DIR, img_file))
    img_list[x].append(img)

img_vs = []
for k, values in img_list.items():
    img_v = cv2.vconcat(values)
    img_vs.append(img_v)

img = cv2.hconcat(img_vs)
cv2.imwrite(f"{PARENT_DIR}/{IMG_NAME}_mask.png", img)

100%|██████████| 176/176 [00:00<00:00, 404.01it/s]


True

In [17]:
print(len(img_list))
print(img_list['219884'][0])

11
(512, 512, 3)


### 1.2 In case we already have prediction tif file

Convert .tif file to .png file

In [ ]:
img = cv2.imread(f'{PARENT_DIR}/path_to_tif_file.tif', 0)
cv2.imwrite(f"{PARENT_DIR}/{IMG_NAME}_mask.png", img)

## 2. Convert Image mask to GeoJson

In [4]:
def lonlat2xy(lon_deg, lat_deg, zoom=18): 
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = ((lon_deg + 180.0) / 360.0 * n)
    ytile = ((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
    return (xtile, ytile)

def convertCRS(xtile, ytile, zoom=18):
	n = 2.0 ** zoom
	lon_deg = xtile / n * 360.0 - 180.0
	lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
	lat_deg = math.degrees(lat_rad)
	return [ lon_deg,lat_deg]
 
def xyToTileXY(xtile, ytile, x, y,tile_size=256):
	x = float(xtile * tile_size + x) / tile_size
	y = float(ytile * tile_size + y) / tile_size
	# print(x, y)
	return x, y
 
def geojson_xy2latlon(geojson, minx, miny,class_label, zoom_level=18,tile_size=256):
	feature_collection = geojson  # json.loads(geojson)
	for i in range(len(feature_collection['features'])):
		feature_collection['features'][i]["properties"]={"class_label": class_label}
		for j in range(len(feature_collection['features'][i]['geometry']["coordinates"])):			
			for k in range(len(feature_collection['features'][i]['geometry']["coordinates"][j])):
				ind1, ind2 = feature_collection['features'][i]['geometry']["coordinates"][j][k]
				ind1, ind2 = xyToTileXY(minx, miny, ind1, ind2,tile_size)
				lat, lon = convertCRS(ind1, ind2, zoom_level)
				feature_collection['features'][i]['geometry']["coordinates"][j][k] = lat, lon
	return feature_collection

### 2.1 Get BoundingBox information in case we don't have tiles information

In [ ]:
MINX,MINY, MAXX, MAXY = 121.9655355969999988,39.0788434480000006, 121.9792265469999961,39.0944099219999970

zoom_level = 18

MINX, MINY = lonlat2xy(MINX, MINY, zoom=zoom_level) 
MAXX, MAXY = lonlat2xy(MAXX, MAXY, zoom=zoom_level) 
MINX,MINY, MAXX, MAXY =  MINX, MINY, MAXX, MAXY
if MINY > MAXY:
    MINY, MAXY = MAXY, MINY
print(MINX,MINY, MAXX, MAXY)

### 2.2 Get BoudingBox information from tiles x and y

In [5]:
MINX, MINY, MAXX, MAXY = 223564, 101795, 223571, 101801

### 2.3 Convert Image Mask to GeoJSON

In [8]:
# Conversion to geojson
output_image = cv2.imread(f'{PARENT_DIR}/{IMG_NAME}_mask.png', 0)
output_image = ~output_image

cv2.imwrite(
    PARENT_DIR + f"/{IMG_NAME}_invert_mask.ppm",
    cv2.cvtColor(cv2.flip(output_image, 0), cv2.COLOR_GRAY2BGR),
)

order = "-O 10 -a 0 -u 100 -t 10 --longcurve"
sys_command = f"potrace -b geojson {PARENT_DIR}/{IMG_NAME}_invert_mask.ppm  {order} -o {PARENT_DIR}/{IMG_NAME}_mask.geojson"
os.system(sys_command)


with open(PARENT_DIR + f"/{IMG_NAME}_mask.geojson", 'r') as f:
    data = json.load(f)
    
latlon_data = geojson_xy2latlon(data,MINX,MINY,"BULDING", 18, 2048)

with open(PARENT_DIR + f"/{IMG_NAME}_latlon.geojson", 'w') as json_file:
  json.dump(latlon_data, json_file)

sh: 1: potrace: not found


32512

In [ ]:
# Conversion to geojson
from PIL import Image 

output_image = Image.open(f'{PARENT_DIR}/{IMG_NAME}_mask.png').convert("L")
output_image = ~output_image

output = np.flipud()
cv2.imwrite(
    PARENT_DIR + f"/{IMG_NAME}_invert_mask.ppm",
    cv2.cvtColor(cv2.flip(output_image, 0), cv2.COLOR_GRAY2BGR),
)

order = "-O 10 -a 0 -u 100 -t 10 --longcurve"
sys_command = f"potrace -b geojson {PARENT_DIR}/{IMG_NAME}_invert_mask.ppm  {order} -o {PARENT_DIR}/{IMG_NAME}_mask.geojson"
os.system(sys_command)


with open(PARENT_DIR + f"/{IMG_NAME}_mask.geojson", 'r') as f:
    data = json.load(f)
    
latlon_data = geojson_xy2latlon(data,MINX,MINY,"BULDING", 18, 2048)

with open(PARENT_DIR + f"/{IMG_NAME}_latlon.geojson", 'w') as json_file:
  json.dump(latlon_data, json_file)

## 3. Convert JSON File to String

We need to convert json data to string -> copy string to postman for testing

In [12]:
if __name__ == '__main__':
    with open(PARENT_DIR + f"/{IMG_NAME}_latlon.geojson", 'r') as json_file:
        data = json.load(json_file)

    data_str = str(data)
    data_str = data_str.replace("'", "\\\"")
    data_str = f"\"{data_str}\""
    print(data_str)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'class_label': 'BULDING'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[121.96569263935089, 39.07884257204928], [121.96560010313988, 39.078842509585], [121.96557328104973, 39.07884307176366], [121.96554645895958, 39.07884365476372], [121.96554310619831, 39.0788471319427], [121.96553975343704, 39.07885058830008], [121.96553975343704, 39.07887515758253], [121.96553975343704, 39.07889972685643], [121.96554511785507, 39.07890232953325], [121.9655504822731, 39.07890493220997], [121.9655504822731, 39.07890730585106], [121.9655504822731, 39.07890970031349], [121.96556483209133, 39.07891513470181], [121.96557918190956, 39.07892058991112], [121.96558360755444, 39.07892058991112], [121.96558803319931, 39.07892058991112], [121.96558803319931, 39.07892267205197], [121.96558803319931, 39.07892475419275], [121.96559339761734, 39.07892475419275], [121.96559876203537, 39.07892475419275], [121.96559876203537, 39.07892787